In [1]:
%run -i bestestimator_v2.py

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Multiclass

In [2]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Train.csv", sep = ',')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target.csv", sep = ';')

In [3]:
clf = BestEstimator(type_esti = 'classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = True,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 50000,
            n_grid = 50000,
            value_nan = 0,
            view_nan = True,
           params = False,
        scoring = 'AUC') 

Missing Values :

                      Total       %
WARRANTIES_PRICE      96603  96.603
SHIPPING_PRICE        67610  67.610
BUYER_BIRTHDAY_DATE    5836   5.836
SHIPPING_MODE           315   0.315
PRICECLUB_STATUS         57   0.057
SELLER_SCORE_AVERAGE      6   0.006
SELLER_SCORE_COUNT        6   0.006 

NaN data filled by 0 

Searching for the best regressor on 50000 data using AUC loss... 


 Bagging: 0.5668 (+/- 0.0023)

 Gradient Boosting: 0.5479 (+/- 0.0008)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



 XGBoost: 0.5346 (+/- 0.0078)

 Random Forest: 0.5685 (+/- 0.0049)

 Decision Tree: 0.5607 (+/- 0.0027)

 Extra Tree: 0.5636 (+/- 0.0078)

 KNN: 0.5286 (+/- 0.0030)

 SVM: 0.5221 (+/- 0.0024)

 Searching for the best hyperparametres of Random Forest using hard_grid on 50000 data among : 

{'n_estimators': [10, 100, 300, 600, 1000, 1200], 'max_depth': [5, 10, 15, 20, 25], 'criterion': ['gini', 'entropy']} 

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed: 43.8min finished



 Finally, the best estimator is : Random Forest classifier

 Using these hyperparametres : {'criterion': 'gini', 'max_depth': 25, 'n_estimators': 100}

 With this AUC score : 0.57062244392951


In [4]:
clf.pred_grid(Train[0:50], ID = 'ID', prob = True)

,ID,-,DAMAGED,DIFFERENT,FAKE,NOT_RECEIVED,SELLER_CANCEL_POSTERIORI,UNDEFINED,WITHDRAWAL
0,0,0.430889,0.055665,0.061617,0.00,0.106667,0.235162,0.010000,0.100000
1,1,0.580000,0.025000,0.030000,0.00,0.045000,0.190000,0.050000,0.080000
2,2,0.340000,0.030000,0.080000,0.00,0.305000,0.185000,0.020000,0.040000
3,3,0.260000,0.020000,0.050000,0.01,0.250000,0.240000,0.060000,0.110000
4,4,0.195000,0.070769,0.090000,0.03,0.186154,0.180000,0.126923,0.121154
5,5,0.320000,0.070000,0.000000,0.00,0.150000,0.280000,0.090000,0.090000
6,6,0.362143,0.020000,0.060000,0.00,0.190000,0.152857,0.080000,0.135000
7,7,0.340000,0.080000,0.030000,0.00,0.110000,0.130000,0.070000,0.240000
8,8,0.440000,0.090000,0.050000,0.00,0.120000,0.160000,0.040000,0.100000
9,9,0.495000,0.040000,0.000000,0.00,0.160000,0.150000,0.060000,0.095000


In [5]:
clf.pred(Train[0:50], ID = 'ID', prob = True, n = 10000)

bestestimator_v2.py:431: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estim = self.Decision_Function.fit(train, target)


,ID,-,DAMAGED,DIFFERENT,FAKE,NOT_RECEIVED,SELLER_CANCEL_POSTERIORI,UNDEFINED,WITHDRAWAL
0,0,0.402857,0.090000,0.121429,0.00,0.160000,0.165000,0.040714,0.020000
1,1,0.350000,0.050000,0.030000,0.00,0.180000,0.280000,0.040000,0.070000
2,2,0.180000,0.120000,0.010000,0.00,0.480000,0.120000,0.030000,0.060000
3,3,0.300000,0.073333,0.060000,0.00,0.156667,0.270000,0.060000,0.080000
4,4,0.200000,0.120000,0.040000,0.01,0.220000,0.150000,0.110000,0.150000
5,5,0.520000,0.050000,0.060000,0.00,0.110000,0.150000,0.070000,0.040000
6,6,0.300000,0.060000,0.040000,0.00,0.160000,0.240000,0.150000,0.050000
7,7,0.320000,0.030000,0.080000,0.00,0.110000,0.220000,0.030000,0.210000
8,8,0.410000,0.120000,0.080000,0.00,0.070000,0.190000,0.030000,0.100000
9,9,0.540000,0.010000,0.050000,0.00,0.150000,0.160000,0.030000,0.060000


### Binary

In [6]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Train1.csv", sep = ';')
target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target1.csv", sep = ';')

Target = pd.DataFrame()
Target['ID'] = target['ID']
Target['Target'] = [np.nan]*target.shape[0]


for i in range(target.shape[0]):
    if target['Target'][i] == 1:
        Target['Target'][i] ='+'
    else :
        Target['Target'][i] ='-'
        
        

bestestimator_v2.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
clf = BestEstimator(type_esti = 'classifier', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = True,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 50000,
            n_grid = 50000,
            value_nan = 0,
            view_nan = True,
           params = False,
        scoring = 'AUC') 

Missing Values :

                            Total         %
1_diffClosing stocks(kmt)     261  2.569151
2_diffClosing stocks(kmt)     257  2.529777
3_diffClosing stocks(kmt)     253  2.490403
4_diffClosing stocks(kmt)     249  2.451029
5_diffClosing stocks(kmt)     245  2.411655
6_diffClosing stocks(kmt)     241  2.372281
7_diffClosing stocks(kmt)     238  2.342750
8_diffClosing stocks(kmt)     235  2.313220
9_diffClosing stocks(kmt)     232  2.283689
10_diffClosing stocks(kmt)    228  2.244315
11_diffClosing stocks(kmt)    224  2.204941
12_diffClosing stocks(kmt)    220  2.165567
1_diffImports(kmt)            151  1.486367
2_diffImports(kmt)            148  1.456836
3_diffImports(kmt)            145  1.427306
4_diffImports(kmt)            142  1.397775
5_diffImports(kmt)            139  1.368245
6_diffImports(kmt)            136  1.338714
7_diffImports(kmt)            133  1.309184
8_diffImports(kmt)            130  1.279654
9_diffImports(kmt)            127  1.250123
10_diffImports

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



 XGBoost: 0.6846 (+/- 0.0151)

 Random Forest: 0.6859 (+/- 0.0108)

 Decision Tree: 0.6172 (+/- 0.0158)

 Extra Tree: 0.6516 (+/- 0.0161)

 KNN: 0.5738 (+/- 0.0096)

 SVM: 0.4998 (+/- 0.0006)

 Searching for the best hyperparametres of Random Forest using hard_grid on 50000 data among : 

{'n_estimators': [10, 100, 300, 600, 1000, 1200], 'max_depth': [5, 10, 15, 20, 25], 'criterion': ['gini', 'entropy']} 

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed: 17.7min finished



 Finally, the best estimator is : Random Forest classifier

 Using these hyperparametres : {'criterion': 'gini', 'max_depth': 25, 'n_estimators': 1200}

 With this AUC score : 0.6939601783831147


In [8]:
clf.pred_grid(Train[0:50], ID = 'ID', prob = True)

,ID,+,-
0,ID00001,0.950000,0.050000
1,ID00002,0.258637,0.741363
2,ID00003,0.020980,0.979020
3,ID00004,0.138541,0.861459
4,ID00005,0.871694,0.128306
5,ID00006,0.707709,0.292291
6,ID00007,0.085968,0.914032
7,ID00008,0.019450,0.980550
8,ID00009,0.292775,0.707225
9,ID00010,0.137649,0.862351


In [9]:
clf.pred(Train[0:50], ID = 'ID', prob = True, n = 10000)

bestestimator_v2.py:431: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estim = self.Decision_Function.fit(train, target)


,ID,+,-
0,ID00001,0.966087,0.033913
1,ID00002,0.241333,0.758667
2,ID00003,0.091247,0.908753
3,ID00004,0.127689,0.872311
4,ID00005,0.875667,0.124333
5,ID00006,0.722045,0.277955
6,ID00007,0.095983,0.904017
7,ID00008,0.010661,0.989339
8,ID00009,0.298429,0.701571
9,ID00010,0.060199,0.939801


### Regression

In [10]:
Train = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\input_training.csv", sep = ';')
Target = pd.read_csv(r"C:\Users\jecombe\OneDrive - Capgemini\Notebooks\Target_Engie.csv", sep = ';')

In [ ]:
clf = BestEstimator(type_esti = 'regressor', 
                 cv = 3, 
                 grid = True, 
                 hard_grid = True,
                 cv_grid = 3)

clf.fit(Train, Target,
            ID = 'ID',
            target_ID = True,
            n = 50000,
            n_grid = 50000,
            value_nan = 0,
            view_nan = True,
           params = False,
        scoring = 'r2') 

Missing Values :

                                Total          %
Grid_voltage                   101322  16.411451
Grid_voltage_std               101322  16.411451
Grid_voltage_max               101322  16.411451
Grid_voltage_min               101322  16.411451
Gearbox_inlet_temperature        8064   1.306152
Generator_converter_speed        8064   1.306152
Generator_converter_speed_min    8064   1.306152
Generator_converter_speed_max    8064   1.306152
Generator_converter_speed_std    8064   1.306152
Gearbox_inlet_temperature_min    8064   1.306152
Gearbox_inlet_temperature_max    8064   1.306152
Gearbox_inlet_temperature_std    8064   1.306152
Absolute_wind_direction_c          72   0.011662
Nacelle_angle_c                    72   0.011662 

NaN data filled by 0 

Searching for the best regressor on 50000 data using r2 loss... 


 Bagging: 0.9908 (+/- 0.0004)

 Gradient Boosting: 0.9898 (+/- 0.0011)

 XGBoost: 0.9897 (+/- 0.0007)

 Random Forest: 0.9918 (+/- 0.0003)

 Decision Tree:

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 61.6min


In [ ]:
clf.pred_grid(Train[0:50], ID = 'ID', prob = False)

In [ ]:
clf.pred(Train[0:50], ID = 'ID', prob = False, n = 10000)